In [1]:
import numpy as np
seed = 7
np.random.seed(seed)

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

Using Theano backend.


In [3]:
from keras import optimizers

In [4]:
import keras.backend as K

In [5]:
def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [6]:
def specificity(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)
    
    spesifisitas=(tn/(fp+tn))
    return spesifisitas

In [8]:
from keras.models import load_model

In [9]:
import pandas as pd
df = pd.read_csv("wbc.csv")

In [10]:
df=df.replace('?',np.nan)
df['bare_nuclei'] = pd.to_numeric(df.bare_nuclei.fillna(df.bare_nuclei.median()))

In [11]:
X = df.drop([df.columns[-1]], axis=1)
y = df.jenis.map({2: 0, 4: 1})

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.33, random_state=seed )

In [13]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

In [14]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

In [15]:
cvscores = []
cvscores2 = []
cvscores3 = []

In [17]:
for train in kfold.split(X_train, y_train):
  # create model
    model = Sequential()
    model.add(Dense(10, input_dim=9, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    adm = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='binary_crossentropy', optimizer=adm, metrics=['accuracy',recall,specificity])
    model.save_weights('my_model_weights.h5')
    model.load_weights('my_model_weights.h5')
    model.fit(X_train.as_matrix(), y_train, epochs=10, class_weight = {0: 1.0, 1: 8}, verbose=0)

    scores = model.evaluate(X_train.as_matrix(), y_train)
    print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
    print("%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
    print()
    print()
    
    cvscores.append(scores[1] * 100)
    cvscores2.append(scores[2] * 100)
    cvscores3.append(scores[3] * 100)

print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores2), np.std(cvscores2)))   
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores3), np.std(cvscores3)))   

 32/468 [=>............................] - ETA: 0s
acc: 97.86%
recall: 100.00%
specificity: 96.96%


 32/468 [=>............................] - ETA: 0s
acc: 96.58%
recall: 100.00%
specificity: 94.87%


 32/468 [=>............................] - ETA: 0s
acc: 97.22%
recall: 100.00%
specificity: 95.72%


 32/468 [=>............................] - ETA: 0s
acc: 97.44%
recall: 100.00%
specificity: 96.05%


 32/468 [=>............................] - ETA: 0s
acc: 98.50%
recall: 100.00%
specificity: 97.86%


 32/468 [=>............................] - ETA: 0s
acc: 97.65%
recall: 100.00%
specificity: 96.58%


 32/468 [=>............................] - ETA: 0s
acc: 98.08%
recall: 100.00%
specificity: 97.08%


 32/468 [=>............................] - ETA: 0s
acc: 97.22%
recall: 98.58%
specificity: 96.41%


 32/468 [=>............................] - ETA: 0s
acc: 98.29%
recall: 99.32%
specificity: 97.83%


 32/468 [=>............................] - ETA: 0s
acc: 97.86%
recall: 100.00%
specificity: 9